In [ ]:
!pip install transformers accelerate plotly pandas huggingface_hub


In [ ]:
import getpass
from huggingface_hub import login

token = getpass.getpass("Enter your Hugging Face token: ")
login(token)


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "ibm-granite/granite-3.3-8b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)


In [ ]:
def generate_response(prompt, chat_history=None, temperature=0.2, max_tokens=200):
    if chat_history is None:
        chat_history = []
    # Format history as messages
    full_prompt = ""
    for turn in chat_history:
        full_prompt += f"user: {turn['user']}\nassistant: {turn['bot']}\n"
    full_prompt += f"user: {prompt}\nassistant:"

    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract just the new response
    response = generated_text[len(full_prompt):].strip()
    return response


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

sentiment_pipe = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")


In [ ]:
import pandas as pd

chat_history = []
sentiment_log = []

# Example citizen queries
user_inputs = [
    "How can I report garbage not being collected in my area?",
    "Why is my water supply irregular these days?",
    "The government service was excellent today!",
    "The roads are full of potholes, and nobody is fixing them.",
]

for question in user_inputs:
    response = generate_response(question, chat_history)
    chat_history.append({"user": question, "bot": response})

    sentiment = sentiment_pipe(question)[0]
    sentiment_log.append({"text": question, "label": sentiment["label"], "score": round(sentiment["score"], 2)})

# Display final conversation
for entry in chat_history:
    print(f"👤 User: {entry['user']}\n🤖 Bot: {entry['bot']}\n")


In [ ]:
import plotly.express as px

df = pd.DataFrame(sentiment_log)
fig = px.histogram(df, x="label", color="label", title="Citizen Feedback Sentiment Overview", text_auto=True)
fig.show()


👤 Question: What is the process to apply for a birth certificate?
🤖 Answer: ❌ Error 404: Not Found
